In [1]:
import os
from PIL import Image
from IPython.display import display
from ipywidgets import VBox
import ipywidgets as widgets
import pandas as pd
from ipycanvas import MultiCanvas
import io
import numpy as np
import cv2 

# 폴더 경로 !! 
image_folder = '/home/ircv/projects/deep_daiv/0115/scene/scene-0017'
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]
scene_number = image_folder.split('-')[-1]  

print(f"Loaded {len(image_files)} images.")
canvas = MultiCanvas(2, width=224, height=224, sync_image_data=True, layout=widgets.Layout(width='224px', height='224px'))

current_index = 0
collected_data = []


skip_button = widgets.Button(description="Skip", button_style='info')

def show_image(image_path):
    with open(image_path, 'rb') as file:
        image = Image.open(file).resize((224, 224)).convert('RGB')
        image_array = np.array(image)  
        canvas[0].put_image_data(image_array, 0, 0)


def on_click(x, y):
    global current_index
    image_path = image_files[current_index]
    
    # 클릭한 곳 좌표 저장 
    collected_data.append({'image_path': image_path, 'x': x, 'y': y})
    print(f"Saved: {image_path}, x: {x}, y: {y}")
    
    canvas[1].clear()
    canvas[1].fill_style = 'blue'
    canvas[1].fill_circle(x, y, 5)
    
    save_annotated_image(current_index, x, y)
    
    current_index += 1
    if current_index < len(image_files):
        show_image(image_files[current_index])
    else:
        print("All images labeled!")
        save_data()
        canvas[0].clear()  
        canvas[1].clear()


def on_skip(button):
    global current_index
    current_index += 1
    if current_index < len(image_files):
        print(f"Skipped: {image_files[current_index - 1]}")
        show_image(image_files[current_index])
    else:
        print("All images labeled!")
        save_data()
        canvas[0].clear()
        canvas[1].clear()


def save_annotated_image(index, x, y):
    parent_folder = "annotated_images"
    output_folder = os.path.join(parent_folder, f"annotated_images_{scene_number}")
    os.makedirs(output_folder, exist_ok=True)
    
    image_path = image_files[index]
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  
    
    cv2.circle(image, (int(x), int(y)), 5, (255, 0, 0), -1)
    
    output_path = os.path.join(output_folder, f"annotated_image_{index}.jpg")
    cv2.imwrite(output_path, image)
    print(f"Annotated image saved to {output_path}")
    

def save_data():
    parent_folder = "label"  
    os.makedirs(parent_folder, exist_ok=True)  
    
    label_filename = os.path.join(parent_folder, f"label_{scene_number}.csv")
    df = pd.DataFrame(collected_data)
    df.to_csv(label_filename, index=False)
    print(f"Data saved to {label_filename}")


canvas.on_mouse_down(on_click)
skip_button.on_click(on_skip)

show_image(image_files[current_index])

display(VBox([canvas, skip_button]))

Loaded 159 images.
